In [2]:
import numpy as np
from manet.hmc import HMC
from manet.m3n import M3N, one_hot
from manet.mn_models import MrMaNetHomo, AdvMaNetHomo

We will use synthetic data generated from a Hidden Markov Chain of discrete sequences. That is, the observable sequence $(x_0,\ldots,x_{l-1})\in \{0,\ldots,n_x-1\}^l$ and the hidden sequence $(y_0,\ldots,y_{l-1})\in \{0,\ldots,n_y-1\}^l$ follow the distribution:
$$ P(x_0,\ldots,x_{l-1},y_0,\ldots,y_{l-1})=P_0(y_0)\prod_{i=1}^{l-1} P(y_i\mid y_{i-1}) P(x_i\mid y_i)$$

We set the transition distribution to
$$P(y_{i+1}\mid y_i) = \left \{ \begin{array}{lll}
   \alpha & if & y_{i+1} = y_i\\
   (1-\alpha)/(n_y-1) & if & y_{i+1} \neq y_i
  \end{array}
  \right .
$$
and the emission probability to
$$P(x_i\mid y_i) = \left \{ \begin{array}{lll}
   \beta & if & x_i = y_i\\
   (1-\beta)/(n_x-1) & if & x_i \neq y_i
  \end{array}
  \right .
$$

The number of observeble symbols $n_x$, hidden symbols $n_y$, the initial state probability $P_0(y_0)$, the numbers $\alpha\in(0,1)$, $\beta\in(0,1)$ and the sequence length $l$ are hyperparameters defined below.

In [19]:
n_x = 30  # number of observable symbols
n_y = 10  # number of hidden symbols
alpha = 0.7 
beta = 0.7
seq_length = 10 

# define transition probability
trans = (1-beta)*np.ones((n_y,n_y)) / (n_y-1)
np.fill_diagonal( trans, beta )

# define emission probability
emis = (1-alpha)*np.ones( (n_y,n_x )) / (n_x - 1)
np.fill_diagonal( emis, alpha )

# define initial state probability
p0 = np.ones( n_y ) / n_y

# initialize HMC class
Hmc = HMC( p0, trans, emis, alphabet=np.int16)

Because we know the distribution of the data $P(x_0,\ldots,x_{l-1},y_0,\ldots,y_{l-1})$, we can construct Bayes optimal rule provided the goals is to minimize the expected Hamming loss 
$$
  \ell_{Hamming}(y_0,\ldots,y_{l-1},\hat{y}_0,\ldots,\hat{y}_{l-1}) = \frac{1}{l}\sum_{i=1}^{l-1}\delta(y_i\neq \hat{y}_i)
$$
or the expected 0/1-loss
$$
  \ell_{0/1}(y_0,\ldots,y_{l-1},\hat{y}_0,\ldots,\hat{y}_{l-1}) = \left \{
    \begin{array}{ll}
      1 & if \; \exist y_i\neq \hat{y}_i\\
      0 & otherwise
      \end{array}
      \right .
$$
In case of the Hamming loss, the Bayes rule is a sequence of the most probable states which can be computed by function <code>HMC.decode</code>. In case of the 0/1l-loss, the Bayes rule is the maximum a posteriory sequence computed which can be computed by function <code>HMC.map</code>. Having the Bayes rules, we estimate the Bayes risk (best attainable) to get a reference solution for later comparison with the rule learned from examples by M3N algorithm.

In [22]:
m = 10000

risk_hamming_decode = 0  # Expected Hamming loss of max-marginals (decode) rule
risk_01_map = 0          # Expected 0/1-loss of MAP rule
for i in range( m ):
    X, Y = Hmc.generate( seq_length )

    # Maximum aposteriory rule
    Ymap, log_map = Hmc.map(X)
    risk_01_map += int( np.any( Y != Ymap ) )

    # Decode rule i.e. sequnce of the most probable states
    P = Hmc.decode( X )
    Ydec = np.argmax( P, axis = 0)
    risk_hamming_decode += np.count_nonzero( Y-Ydec ) / seq_length
    
risk_01_map /= m
risk_hamming_decode /= m
    
print("Bayes risk for Hamming loss:", risk_hamming_decode)
print("bayes risk for 0/1-loss:", risk_01_map)

Bayes risk for Hamming loss: 0.15646999999999756
bayes risk for 0/1-loss: 0.7467


Use HMC model to generate set of training sequences and testing sequences.

In [23]:
n_trn_examples = 500
n_tst_examples = 500
def generate_sequences( Hmc, length, n_examples):
    examples = []
    for i in range( n_examples):
        X, Y = Hmc.generate(  length )
        examples.append({'X': X, 'Y': Y, 'n_x': Hmc.n_x, 'n_y': Hmc.n_y} )
    return examples        

trn_sequences = generate_sequences( Hmc, length=10, n_examples=n_trn_examples)
tst_sequences = generate_sequences( Hmc, length=10, n_examples=n_tst_examples)

In [24]:
def create_markov_nets( model, sequences ):
    examples = []
    for seq in sequences:
        length = len( seq['X'])        
        edges = np.concatenate(( np.arange(0,length-1).reshape((1,length-1)),
                                 np.arange(1,length).reshape((1,length-1)) ),axis=0)
        examples.append( model( n_y = seq['n_y'],\
                            X = one_hot(seq['n_x'],seq['X']),\
                            Y = seq['Y'],\
                            E = edges, \
                            graph = 'chain') )
    return examples

model = MrMaNetHomo
#model = AdvMaNetHomo

trn_examples = create_markov_nets( model, trn_sequences)
tst_examples = create_markov_nets( model, tst_sequences)

In [25]:
config = {'num_epochs': 100,
          'lr_const': 0.01,
          'lr_exp': -1,
          'eval_obj': 10,
          'verb': True,
          'solver': 'adam'}

algo = M3N( config )

lam = 0
W, obj_val = algo.train( trn_examples, lam )

epoch=0 obj_val=0.9688188856804634
epoch=10 obj_val=0.4894249807526649
epoch=20 obj_val=0.42876403991525086
epoch=30 obj_val=0.4081299799122288
epoch=40 obj_val=0.39791987660025224
epoch=50 obj_val=0.3912414273496622
epoch=60 obj_val=0.38776856863454234
epoch=70 obj_val=0.38529607243497255
epoch=80 obj_val=0.3831610896594044
epoch=90 obj_val=0.38178697353349267
epoch=99 obj_val=0.38103766553834023


In [27]:
trn_hamming_loss, trn_01_loss = algo.eval( W, trn_examples )
trn_risk_hamming_m3n = np.mean(trn_hamming_loss)
trn_risk_01_m3n = np.mean(trn_01_loss)

print("Training risk of M3N predictor")
print( "Hamming loss:",trn_risk_hamming_m3n)
print( "01-loss:", trn_risk_01_m3n )

tst_hamming_loss, tst_01_loss = algo.eval( W, tst_examples )
tst_risk_hamming_m3n = np.mean(tst_hamming_loss)
tst_risk_01_m3n = np.mean(tst_01_loss)

print()
print("Test risk of M3N predictor")
print( "Hamming loss:",tst_risk_hamming_m3n)
print( "01-loss:", tst_risk_01_m3n)

print()
print("Test risk of Bayes predictor")
print( "Hamming loss:", risk_hamming_decode)
print( "01 loss:", risk_01_map)

Training risk of M3N predictor
Hamming loss: 0.16539999999999996
01-loss: 0.784

Test risk of M3N predictor
Hamming loss: 0.1772
01-loss: 0.812

Test risk of Bayes predictor
Hamming loss: 0.15646999999999756
01 loss: 0.7467


In [31]:
for i in range(10):
    Y_pred, score = algo.predict( W, tst_examples[i] )

    print("example:", i)
    print("predictor score   :", score )
    print("input sequnce     :", tst_sequences[i]['X'])
    print("true sequence     :", tst_sequences[i]['Y'] )
    print("predicted sequence:",Y_pred )

example: 0
predictor score   : 2.956114103988624
input sequnce     : [18  2 25  3  2  8  9  5  5  5]
true sequence     : [2 2 2 2 2 8 9 5 5 5]
predicted sequence: [2 2 3 3 2 8 9 5 5 5]
example: 1
predictor score   : 3.552639619466107
input sequnce     : [ 1  1  1  1  2  2 15  7  7  7]
true sequence     : [1 1 1 1 2 2 2 7 7 7]
predicted sequence: [1 1 1 1 2 2 2 7 7 7]
example: 2
predictor score   : 3.1590741650319796
input sequnce     : [ 9  9  9 28  1 19  7  4  4  2]
true sequence     : [9 9 9 1 1 7 7 4 2 2]
predicted sequence: [9 9 9 1 1 7 7 4 4 2]
example: 3
predictor score   : 3.0384863338625823
input sequnce     : [ 7  7  1 12 20  7  7  8 24  1]
true sequence     : [7 7 7 7 7 7 7 8 5 1]
predicted sequence: [7 7 1 1 1 7 7 8 8 1]
example: 4
predictor score   : 3.1639417626578994
input sequnce     : [ 1  1  8  8  4  8  8  0 19  5]
true sequence     : [1 1 8 8 8 8 8 0 0 0]
predicted sequence: [1 1 8 8 4 8 8 0 5 5]
example: 5
predictor score   : 3.0762611876190893
input sequnce     : [ 